In [1]:
from os import system, name
from math import floor
import random

#Move generator for othello

Othello is a two-player strategy game played on an 8x8 grid with black and white pieces called "discs". The objective is to have more of your color discs on the board than your opponent at the end of the game. Players take turns placing discs on the board, outflanking their opponent's discs to flip them to their color. The game ends when the board is full or neither player can make a move. The player with the most discs of their color on the board at the end of the game wins.

In [2]:
def list_moves(board:list[list[int]])->list[tuple[int, int]]:
    size = len(board)
    moves = []
    for i in range(0, size):
        for j in range(0, size):
            if board[i][j] == 0:
                moves.append((i, j))
    random.shuffle(moves) #gives a random ordering
    return moves

#Other utility functions

In [3]:
#initializes board
def initialize_board(player1, player2, size=8)->list[list[int]]:
    board =  [[0 for i in range(size)] for j in range(size)]
    mid = floor(size/2)
    board[mid][mid-1] = player1
    board[mid-1][mid] = player1
    board[mid][mid] = player2
    board[mid-1][mid-1] = player2
    return board

In [4]:
def count(board:list[list[int]], Player:int)->int: #count of a players pieces on board
    size  = len(board)
    count = 0
    for i in range(size):
        for j in range(size):
            if board[i][j] == Player:
                count += 1
    return count

def print_board(board:list[list[int]]): #print board
    size = len(board)
    print("  ", end="")
    for i in range(size):
        print(i, end=" ")
    print()
    for i in range(size):
        print(i, end=" ")
        for j in range(size):
            print(board[i][j], end=" ")
        print()

def check_valid(board, x, y)->bool: #check if a move is valid (empty cell + in boundaries)
    if x<0 or y<0 or x>=len(board) or y>=len(board):
        return False
    if board[x][y] != 0:
        return False
    return True

def check_over(board)->bool: #checks if game is over
    size = len(board)
    for i in range(size):
        for j in range(size):
            if board[i][j] == 0:
                return False
    return True

#Modify board after a move

In [5]:
def check_modify(board, x, y)->bool: #util function for step. if its an occupied cell it has a chance of being flipped over after a move.
    if x<0 or y<0 or x>=len(board) or y>=len(board):
        return False
    if board[x][y] == 0:
        return False
    return True

def step(board : list[list[int]], Player:int, row:int, col:int): #modifies the board after a move. flips pieces accordingly.
    board[row][col] = Player
    directions = [[1, 0], [0, 1], [1, 1], [-1, 0], [0, -1], [-1, -1], [-1, 1], [1, -1]]
    for dir in directions:
      dr = dir[0]
      dc = dir[1]

      tr = row + dr
      tc = col + dc

      while check_modify(board, tr, tc):
        
        if board[tr][tc] == Player:
          while tc != col or tr != row:
            board[tr][tc] = Player
            tr -= dr
            tc -= dc
          break
        tr += dr
        tc += dc
    return board

#Evaluation functions

In [6]:
def evaluate_simple(board, player):
    player_score = [board[i][j] == player for i in range(len(board)) for j in range(len(board))].count(True)
    opponent_score = len(board)**2 - player_score -[board[i][j] == 0 for i in range(len(board)) for j in range(len(board))].count(True)
    return player_score - opponent_score

In [7]:
def get_valid_moves(board, player):
    size = len(board)
    valid_moves = []
    for r in range(size):
        for c in range(size):
            if board[r][c] == 0:
                for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
                    nr, nc = r + dr, c + dc
                    if nr < 0 or nr >= size or nc < 0 or nc >= size or board[nr][nc] == player:
                        continue
                    while nr >= 0 and nr < size and nc >= 0 and nc < size and board[nr][nc] != 0:
                        if board[nr][nc] == player:
                            valid_moves.append((r, c))
                            break
                        nr += dr
                        nc += dc
    return valid_moves

def evaluate_improved(board, player):

    opponent = 1 if player == 2 else 2
    size = len(board)
    
    # weights for each square on the board
    weights = [[100, -20, 10, 5, 5, 10, -20, 100],
               [-20, -50, -2, -2, -2, -2, -50, -20],
               [10, -2, -1, -1, -1, -1, -2, 10],
               [5, -2, -1, -1, -1, -1, -2, 5],
               [5, -2, -1, -1, -1, -1, -2, 5],
               [10, -2, -1, -1, -1, -1, -2, 10],
               [-20, -50, -2, -2, -2, -2, -50, -20],
               [100, -20, 10, 5, 5, 10, -20, 100]]

    # number of pieces owned by each player
    player_score = sum([board[i][j] == player for i in range(size) for j in range(size)])
    opponent_score = sum([board[i][j] == opponent for i in range(size) for j in range(size)])
    
    # mobility (number of legal moves)
    player_legal_moves = len(get_valid_moves(board, player))
    opponent_legal_moves = len(get_valid_moves(board, opponent))

    # corners (bonus for occupying corner squares)
    player_corners = sum([board[i][j] == player for i, j in [(0, 0), (0, size-1), (size-1, 0), (size-1, size-1)]])
    opponent_corners = sum([board[i][j] == opponent for i, j in [(0, 0), (0, size-1), (size-1, 0), (size-1, size-1)]])

    # weighted sum of features
    player_value = player_score + player_legal_moves + 10 * player_corners# + 100 * player_stability
    opponent_value = opponent_score + opponent_legal_moves + 10 * opponent_corners# + 100 * opponent_stability
    return player_value - opponent_value



#Move ordering
Orders the moves according to a score on the resulting board state.

In [8]:
#arrays to keep track of ebfs every time an agent is run
random_ebf = []
order_ebf = []

def order_moves(board, moves, evaluate, player):
    """
    Order the given list of valid moves for the given Othello board, based on the
    evaluation function.
    """
    scored_moves = []
    for move in moves:
        new_board = step(board, player, move[0], move[1])
        score = evaluate(new_board, player)
        scored_moves.append((move, score))
    # Sort the moves in descending order of score
    scored_moves.sort(key=lambda x: x[1], reverse=True)
    # Return the ordered list of moves
    return [move for move, score in scored_moves]

#Alpha Beta Agent
This implements the Alpha beta search algorithm as given in AIMA.

In [9]:
def max_value(board, depth, alpha, beta, evaluate, player, ordering=None):
    global node_count
    node_count += 1
    if player == 1:
        opponent = 2
    else:
        opponent = 1
    if depth == 0 or check_over(board):
        return evaluate(board, player), None
    max_value = float('-inf')
    best_move = None
    if ordering is not None:
        moves = ordering(board, list_moves(board), evaluate, player)
    else:
        moves = list_moves(board)
    for move in moves:
        new_board = step(board, player, move[0], move[1])
        value, _ = min_value(new_board, depth - 1, alpha, beta, evaluate, opponent, ordering)
        if value > max_value:
            max_value = value
            best_move = move
            alpha = max(alpha, max_value)
        if max_value >= beta:
            break
    return max_value, best_move

def min_value(board, depth, alpha, beta, evaluate, player, ordering=None):
    global node_count
    node_count += 1
    if player == 1:
        opponent = 2
    else:
        opponent = 1
    if depth == 0 or check_over(board):
        return evaluate(board, player), None
    min_value = float('inf')
    best_move = None
    if ordering is not None:
        moves = ordering(board, list_moves(board), evaluate, player)
    else:
        moves = list_moves(board)
    for move in moves:
        new_board = step(board, player, move[0], move[1])
        value, _ = max_value(new_board, depth - 1, alpha, beta, evaluate, opponent, ordering)
        if value < min_value:
            min_value = value
            best_move = move
            beta = min(beta, min_value)
        if min_value <= alpha:
            break
    return min_value, best_move

def agent(board, player, evaluate, depth=16, ordering=None):
    global node_count
    node_count = 0
    global order_ebf
    global random_ebf
    alpha = float('-inf')
    beta = float('inf')
    new_board = [[cell for cell in row] for row in board] 
    _, move = max_value(new_board, depth, alpha, beta, evaluate, player, ordering)
    if ordering is None:
        random_ebf.append((node_count)**(1/depth))
    else:
        order_ebf.append((node_count)**(1/depth))
    return move


#The game
This makes two agents play against each other. By commenting and uncommenting out certain lines of code in the given function, it could be modified to play against a human user as well as print board state after each move. The reason for having two agents play against each other was to compare between different search depths, evaluation functions and assessing how improved move ordering affects the time taken for the agent to make a move.

In [10]:
def game(p1depth, p2depth, p1eval, p2eval, ordering = None):

    player1 = 1
    player2 = 2

    board = initialize_board(player1, player2)
    #print_board(board)

    game_length = len(board)**2-4
    player =1

    for i in range(game_length):

        if check_over(board):
            break
        
        if player == 1:
            #print("Player 1's turn")
            #x = -1
            #y = -1
            #while not check_valid(board, x, y):
            #    x = int(input("Enter row: "))
            #    y = int(input("Enter column: "))
            #step(board, player, int(x), int(y))
            move = agent(board, player, p1eval, p1depth, ordering)
            if move == None:
                break
            row = move[0]
            col = move[1]
            #print(f"Player 1's turn: ({row}, {col})")
            step(board, player, row, col)
        
        elif player == 2:
            move = agent(board, player, p2eval, p2depth, ordering)
            if move == None:
                break
            row = move[0]
            col = move[1]
            #print(f"Player 2's turn: ({row}, {col})")
            step(board, player, row, col)

        #print_board(board)
        #sleep(10)
        #clear()

        if player == 1:
            player = 2
        else:
            player = 1
    
    #print("Final board state:")
    #print_board(board)

    #print("Game Over!")

    p1cnt = [board[i][j] == player1 for i in range(len(board)) for j in range(len(board))].count(True)
    p2cnt = [board[i][j] == player2 for i in range(len(board)) for j in range(len(board))].count(True)
    if p1cnt > p2cnt:
        #print("Player 1 wins!")
        return 1
    elif p2cnt > p1cnt:
        #print("Player 2 wins!")
        return 2
    else:
        #print("It's a tie!")
        return 0
 

#Different Search Depths
Here I tested the game with 2 agents, one with search depth of 10 and the other with 20 and both having the same evaluation function. I tested it on 1000 runs of the game. And the agent with higher search depth performs better. This is due to it having search more possibilities and having a more accurate utility for each move. 
PS: It takes quite a while for 1000 runs. With just 100 or lesser runs I was getting slightly random results.

In [20]:
runs = 1000
p1 = 0
p2 = 0
t = 0
for i in range(runs):
  x = game(10, 20, evaluate_simple, evaluate_simple)
  if x ==0:
    t+=1
  elif x==1:
    p1+=1
  else:
    p2+=1

print(f"Player 1 wins:{p1}")
print(f"Player 2 wins:{p2}")
print(f"Ties:{t}")

Player 1 wins:409
Player 2 wins:550
Ties:41


#Different Evaluation Functions
Here I ran the game with two agents playing against each other. Both searched till a depth of 10. One used a simple evaluation function (evaluate_simple) and the other used a modified version which I expected to capture the game state better (evaluate_improved). The new function basically adds upon the simple difference of piece count used earlier. It gives more importance to corner pieces being occupied by the player, each position now has a weight associated with it. The more unlikely it is to be flipped, the more positive the value of its weight would be. Also, I captured the number of moves each player has which would result in opponent pieces being flipped. I worked with an assumption that this would perform better as an utility function for a non terminal node in the minimax tree. At times it performed better slightly but wasn't significantly outperforming the simpler function either.

In [15]:
runs = 100
p1 = 0
p2 = 0
t = 0
for i in range(runs):
  x = game(10, 10, evaluate_simple, evaluate_improved)
  if x ==0:
    t+=1
  elif x==1:
    p1+=1
  else:
    p2+=1

print(f"Player 1 wins:{p1}")
print(f"Player 2 wins:{p2}")
print(f"Ties:{t}")

Player 1 wins:41
Player 2 wins:55
Ties:4


#With and Without move ordering
The move ordering function is as explained above near its definition. On an average, the agent with move ordering is about 3-4 times faster than the one without(random). This is due to the agent exploring the more preferable branches earlier, thus promoting pruning.
The EBF is calculated as (node_count)^(1/search_depth) as an approximate solution.
On multiple runs, i get the Average EBF of with ordering to be lesser than Average EBF with random ordering. This means that my move ordering has come slightly closer to the case with perfect move ordering. Also, in this case the ordering uses the simpler evaluation function. If we used the improved function, the EBF might come out to be lower.

In [16]:
import timeit
exec_time_no_order = timeit.timeit(lambda: game(10, 10, evaluate_simple, evaluate_simple), number = 10)
exec_time_order = timeit.timeit(lambda: game(10, 10, evaluate_simple, evaluate_simple, order_moves), number = 10)

print(f"Execution time without ordering: {exec_time_no_order}")
print(f"Execution time with ordering: {exec_time_order}")

print("Average EBF without ordering: ", sum(random_ebf)/len(random_ebf))
print("Average EBF with ordering: ", sum(order_ebf)/len(order_ebf))

Execution time without ordering: 2.462466250000034
Execution time with ordering: 0.49449924999998984
Average EBF without ordering:  1.512942751377036
Average EBF with ordering:  1.3831335547696637


#Part d 
I did A* and B* search over the romania map given in AIMA. I used the time taken and path cost as a metric to compare between the two search strategies.  I took all possible combinations of start and goal cities and ran both algorithms on it. I used the straight line distance heuristic for the same.

The A* search algorithm evaluates nodes based on two cost functions: the cost to reach the node from the start node (g(n)) and the estimated cost to reach the goal node from the current node (h(n))(heuristic). The total cost of a node is the sum of these two costs (f(n) = g(n) + h(n)). The algorithm then chooses the node with the lowest f-value as the next node to expand.

B* search is similar to A* search, but uses the estimated cost as a backup value instead of the primary heuristic function. Nodes are evaluated based on the actual cost and estimated cost to reach the goal. The backup value used in B* search is the difference between the heuristic value of the node's parent and the current node's heuristic value. B* search is useful when the heuristic function is unreliable, but requires more computational resources than A* search.

In [17]:
import heapq
import math
import timeit
import itertools

#romania map as given in AIMA
romania_map = {
    'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Pitesti': {'Rimnicu Vilcea': 97, 'Craiova': 138, 'Bucharest': 101},
    'Bucharest': {'Pitesti': 101, 'Fagaras': 211, 'Giurgiu': 90, 'Urziceni': 85},
    'Giurgiu': {'Bucharest': 90},
    'Urziceni': {'Bucharest': 85, 'Vaslui': 142, 'Hirsova': 98},
    'Hirsova': {'Urziceni': 98, 'Eforie': 86},
    'Eforie': {'Hirsova': 86},
    'Vaslui': {'Urziceni': 142, 'Iasi': 92},
    'Iasi': {'Vaslui': 92, 'Neamt': 87},
    'Neamt': {'Iasi': 87}
}

#function to calculate the straight line distance (heuristic) between two cities
def straight_line_distance_heuristic(city, goal_city):
    locations = {
        'Arad': (91, 492),
        'Bucharest': (400, 327),
        'Craiova': (253, 288),
        'Drobeta': (165, 299),
        'Eforie': (562, 375),
        'Fagaras': (305, 449),
        'Giurgiu': (375, 270),
        'Hirsova': (534, 350),
        'Iasi': (623, 510),
        'Lugoj': (111, 410),
        'Mehadia': (123, 363),
        'Neamt': (677, 547),
        'Oradea': (131, 571),
        'Pitesti': (320, 368),
        'Rimnicu Vilcea': (233, 410),
        'Sibiu': (207, 457),
        'Timisoara': (75, 410),
        'Urziceni': (456, 350),
        'Vaslui': (715, 455),
        'Zerind': (108, 531)
    }
    x1, y1 = locations[city]
    x2, y2 = locations[goal_city]
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

#find path length
def path_length(path, road_map):
    length = 0
    for i in range(len(path) - 1):
        length += road_map[path[i]][path[i + 1]]
    return length   

#performs A* search on the romania map
def a_star_search(start_city, goal_city, heuristic_func, road_map):
    frontier = [(0, start_city)]
    explored = set()
    parent_map = {start_city: None}
    g_score = {start_city: 0}
    f_score = {start_city: heuristic_func(start_city, goal_city)}

    while frontier:
        current_f_score, current_city = heapq.heappop(frontier)
        if current_city == goal_city:
            path = [goal_city]
            while path[-1] != start_city:
                path.append(parent_map[path[-1]])
            path.reverse()
            return path

        explored.add(current_city)
        for neighbor in road_map[current_city]:
            distance = road_map[current_city][neighbor]
            if neighbor in explored:
                continue

            tentative_g_score = g_score[current_city] + distance
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                parent_map[neighbor] = current_city
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic_func(neighbor, goal_city)
                heapq.heappush(frontier, (f_score[neighbor], neighbor))

    return None

#performs B* search on the romania map
import heapq

def b_star_search(start_city, goal_city, heuristic_func, road_map):
    frontier = [(0, start_city)]
    explored = set()
    parent_map = {start_city: None}
    g_score = {start_city: 0}

    while frontier:
        current_f_score, current_city = heapq.heappop(frontier)
        if current_city == goal_city:
            path = [goal_city]
            while path[-1] != start_city:
                path.append(parent_map[path[-1]])
            path.reverse()
            return path

        explored.add(current_city)
        for neighbor in road_map[current_city]:
            distance = road_map[current_city][neighbor]
            if neighbor in explored:
                continue

            new_g_score = g_score[current_city] + distance
            if neighbor not in g_score or new_g_score < g_score[neighbor]:
                parent_map[neighbor] = current_city
                g_score[neighbor] = new_g_score
                f_score = new_g_score + heuristic_func(neighbor, goal_city) + heuristic_func(start_city, neighbor)
                heapq.heappush(frontier, (f_score, neighbor))

    return None


 
def run(start_city, goal_city):

    #test A* search
    a_star_path = a_star_search(start_city, goal_city, straight_line_distance_heuristic, romania_map)

    if a_star_search is None:
        print('No path found from {} to {}'.format(start_city, goal_city))
    path_len_a_star = path_length(a_star_path, romania_map)
    print('A* path from {} to {}: {}'.format(start_city, goal_city, a_star_path))
    print('A* path length: {}'.format(path_len_a_star))
    execution_time = timeit.timeit(lambda: a_star_search(start_city, goal_city, straight_line_distance_heuristic, romania_map), number=1)
    time_a_star = execution_time
    print('Execution time A* search: {}'.format(execution_time))

    #Test B* search
    b_star_path = b_star_search(start_city, goal_city, straight_line_distance_heuristic, romania_map)

    if b_star_search is None:
        print('No path found from {} to {}'.format(start_city, goal_city))
    path_len_b_star = path_length(b_star_path, romania_map)
    print('B* path from {} to {}: {}'.format(start_city, goal_city, b_star_path))
    print('B* path length: {}'.format(path_len_b_star))

    execution_time = timeit.timeit(lambda: b_star_search(start_city, goal_city, straight_line_distance_heuristic, romania_map), number=1)
    time_b_star = execution_time
    print('Execution time B* search: {}'.format(execution_time))

    times = 1 if time_b_star <= time_a_star else 0
    paths = 1 if path_len_b_star < path_len_a_star else 0

    return (paths, times)

#Results
As seen in the results of the code  block, even in multiple runs due to some randomness the number of times when B* is faster does fluctuate a bit, but it never gives a more optimal path than A*.

In [18]:
n  = 0
times = 0
paths = 0
for start_city, goal_city in itertools.combinations(romania_map.keys(), 2):
    n += 1
    ret = run(start_city, goal_city)
    paths += ret[0]
    times += ret[1]

print(f"Total runs: {n}")
print(f"Times B* was faster or took equal amount of time: {times}")
print(f"Times B* gave a shorter path: {paths}")

A* path from Arad to Zerind: ['Arad', 'Zerind']
A* path length: 75
Execution time A* search: 7.3330000986970845e-06
B* path from Arad to Zerind: ['Arad', 'Zerind']
B* path length: 75
Execution time B* search: 7.792000019435363e-06
A* path from Arad to Oradea: ['Arad', 'Zerind', 'Oradea']
A* path length: 146
Execution time A* search: 7.124999910956831e-06
B* path from Arad to Oradea: ['Arad', 'Zerind', 'Oradea']
B* path length: 146
Execution time B* search: 9.416000011697179e-06
A* path from Arad to Sibiu: ['Arad', 'Sibiu']
A* path length: 140
Execution time A* search: 5.209000050854229e-06
B* path from Arad to Sibiu: ['Arad', 'Sibiu']
B* path length: 140
Execution time B* search: 9.08399999843823e-06
A* path from Arad to Fagaras: ['Arad', 'Sibiu', 'Fagaras']
A* path length: 239
Execution time A* search: 9.249999948224286e-06
B* path from Arad to Fagaras: ['Arad', 'Sibiu', 'Fagaras']
B* path length: 239
Execution time B* search: 1.6291999941131508e-05
A* path from Arad to Timisoara: ['A